In [1]:
# CIFAR-10 Dataset has 60000 images of 32x32 with 3 color channels, 10 classes.

import numpy as np

import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from pathlib import Path

Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train.shape, y_train.shape, X_test.shape

170500096/170498071 [==============================] - 11s 0us/step


((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3))

In [0]:
# Normalize data set to 0-to-1 range for pixel value [0...255].
X_train = X_train / 255
X_test = X_test / 255

num_labels = len(np.unique(y_test))
# Convert class vectors to binary class matrices: labels from 0 to 9 to matrix 
# where each label will be array with on element set to 1 and the rest set to 0.
y_train = keras.utils.to_categorical(y_train, num_labels)
y_test = keras.utils.to_categorical(y_test, num_labels)
print(X_train.max())
y_train.shape, y_test.shape

1.0


((50000, 10), (10000, 10))

Create a model and add layers:

In [0]:
size_px = 32
ncolors = 3

model = Sequential()

# Analyze 3x3 tiles and add padding pixels to tile: "same".
model.add(Conv2D(32, (3,3), padding='same', activation='relu',
                 input_shape=(size_px,size_px,ncolors)))
model.add(Conv2D(32, (3,3), activation='relu'))
# Keep only significant parts.
model.add(MaxPooling2D((2, 2)))
# Regularization by cutting some random connections between layers.
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Flatten the input from 2D shape.
model.add(Flatten())

# Regularization by cutting some random connections between layers.
model.add(Dropout(0.01))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)         

In [0]:
# Compile the model with Adaptive Moment Estimation.
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [0]:
# This take 1 hour on CPU!!!

%%time
# Wall time: 1h 16min 1s    in Google Colab.
model.fit(X_train, y_train, batch_size=64, epochs=20,
          validation_data=(X_test, y_test), shuffle=True)

print('Saving model...')
with Path('model_structure.json') as out:
    out.write_text(model.to_json())

print('Saving weights...')
model.save_weights('model_weights.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 228s 5ms/step - loss: 1.2405 - accuracy: 0.5563 - val_loss: 1.0150 - val_accuracy: 0.6394
Epoch 2/20
50000/50000 [==============================] - 230s 5ms/step - loss: 1.0281 - accuracy: 0.6359 - val_loss: 0.9112 - val_accuracy: 0.6765
Epoch 3/20
50000/50000 [==============================] - 230s 5ms/step - loss: 0.8948 - accuracy: 0.6844 - val_loss: 0.7903 - val_accuracy: 0.7239
Epoch 4/20
50000/50000 [==============================] - 229s 5ms/step - loss: 0.8145 - accuracy: 0.7131 - val_loss: 0.7373 - val_accuracy: 0.7412
Epoch 5/20
50000/50000 [==============================] - 230s 5ms/step - loss: 0.7571 - accuracy: 0.7344 - val_loss: 0.7106 - val_accuracy: 0.7505
Epoch 6/20
50000/50000 [==============================] - 230s 5ms/step - loss: 0.7101 - accuracy: 0.7513 - val_loss: 0.7010 - val_accuracy: 0.7610
Epoch 7/20
50000/50000 [==============================] - 230s

In [4]:
from keras.models import model_from_json

# These are the CIFAR10 class labels from the training data (from 0 to 9).
class_labels = [
    "Plane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Boat",
    "Truck"
]

from google.colab import drive
drive.mount('/gdrive')
%cd '/gdrive/My Drive/Colab Notebooks'
!ls

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks
Autograd.ipynb
Autograd_with_tensors.ipynb
car.png
cat.png
complete_neural_net_with_Keras.ipynb
CPU_to_GPU.ipynb
frog.png
model_structure.json
model_weights.h5
PyTorch_FMNIST_Fashion_dataset_predicting.ipynb
PyTorch_Playground.ipynb
PyTorch_Tensors.ipynb
Troubleshooting.ipynb
Using_Optimizers.ipynb
Validation.ipynb


In [5]:
# Load the json file that contains the model's structure.
f = Path('model_structure.json')
model_structure = f.read_text()
# Recreate the Keras model object from the json data.
model = model_from_json(model_structure)

# Re-load the model's trained weights.
model.load_weights('model_weights.h5')

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 6, 6, 64)         

In [6]:
from keras.preprocessing import image

# Load an image file to test, resizing it to 32x32 pixels (required by the model)
img = image.load_img('frog.png', target_size=(32, 32))

# Convert the image to a numpy array.
image_to_test = image.img_to_array(img) / 255

# Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
images_lst = np.expand_dims(image_to_test, axis=0)

# Make a prediction using the model.
preds = model.predict(images_lst)

# Since we are only testing one image, we only need to check the first result.
single_pred = preds[0]

# We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
max_prob_class_idx = int(np.argmax(single_pred))
class_likelihood = single_pred[max_prob_class_idx]

# Get the name of the most likely class.
class_label = class_labels[max_prob_class_idx]

print('This is image is a {} - Likelihood: {:2f}'.format(class_label, class_likelihood))
# This is image is a Cat - Likelihood: 0.980386
# This is image is a Car - Likelihood: 0.902425
# This is image is a Bird - Likelihood: 0.350181    # 20 epochs are not enough, 30 can do the job

This is image is a Bird - Likelihood: 0.350181


In [11]:
from keras.applications import vgg16

# Load Keras' VGG16 model that was pre-trained against the ImageNet database.
model = vgg16.VGG16()

# Load the image file, resizing it to 224x224 pixels (required by this model).
img = image.load_img('bay.jpg', target_size=(224, 224))

# Convert the image to a numpy array and add a fourth dimension.
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

# Normalize the input image's pixel values to the range used when training the neural network.
x = vgg16.preprocess_input(x)

# Run the image through the deep neural network to make a prediction
preds = model.predict(x)
print(preds.shape)
preds[0][:3]

(1, 1000)


array([7.7008048e-07, 2.2140763e-07, 2.1741250e-06], dtype=float32)

In [12]:
# Look up the names of the predicted classes. Index zero is the results for the first image.
predicted_classes = vgg16.decode_predictions(preds)

print('Top predictions for this image:')

for _imagenet_id, name, likelihood in predicted_classes[0]:
    print('Prediction: {} - {:2f}'.format(name, likelihood))

40960/35363 [==================================] - 0s 0us/step
Top predictions for this image:
Prediction: seashore - 0.395213
Prediction: promontory - 0.326128
Prediction: lakeside - 0.119613
Prediction: breakwater - 0.062801
Prediction: sandbar - 0.045267
